# 1. Fonología

## Objetivo

- Læs alumnæs entenderán que es la fonología y un alfabeto fonético
- Manipularan datasets disponibles en repositorios de Github
- Guardaran estos datasets en una estructura de datos
- Recuperarán información reelevante de esta estructura para resolver una tarea específica

## ¿Qué es la fonología?

- La fonología es una rama de la Lingüística que estudia como los humanos producimos y percibimos el lenguaje
    - Producción: La forma en que producimos el lenguaje
    - Percepción: La forma en que interpretamos el lenguaje

In [ ]:
%%HTML
<iframe width="760" height="415" src="https://www.youtube.com/embed/DcNMCB-Gsn8?controls=1"></iframe>

#### Formas comunes

- Oral-Aural
    - Producción: La boca
    - Percepción: Oidos
- Manual-visual
    - Producción: Manual usando las manos
    - Percepción: Visual
- Manual-Manual
    - Producción: Manual usando las manos
    - Percepción: Manual usando las manos

#### International Phonetic Alphabet (IPA)

- Las lenguas naturales tienen muchos sonidos diferentes por lo que necesitamos una forma de describirlos independientemente de las lenguas
- Por ejemplo: Los sonidos del habla se determinan por los movimientos de la boca necesarios para producirlos
- Las dos grandes categorías: Consonantes y Vocales
- IPA es una representación escrita de los [sonidos](https://www.ipachart.com/) del habla

### Dataset: IPA-dict de open-dict

- Diccionario de palabras para varios idiomas con su representación fonética
- Representación simple, una palabra por renglon con el formato:

```
[PALABRA][TAB][IPA]

Ejemplos
mariguana	/maɾiɣwana/
zyuganov's   /ˈzjuɡɑnɑvz/, /ˈzuɡɑnɑvz/
```

- [Github repo](https://github.com/open-dict-data/ipa-dict)
  - [ISO language codes](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes)
  - URL: `https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/<iso-lang>`

### Encontrando palabras que tengan terminación similar

In [ ]:
from collections import defaultdict

#sentence = "There once was a cat that ate a rat and after that sat on a yellow mat"
#sentence = "the cat sat on the mat and looked at the rat."
#sentence = "If you drop the ball it will fall on the doll"
sentence = "cuando juego con fuego siento como brilla la orilla de mi corazón"

#lang = "en_US"
lang = "es_MX"
words = sentence.split(" ")

# Get words and IPA transciptions map
word_ipa_map = {}
for word in words:
    ipa_transcriptions = get_ipa_transcriptions(word=word, dataset=dataset.get(lang))
    ipa_transcriptions = [_.strip("/") for _ in ipa_transcriptions]
    word_ipa_map.update({word: ipa_transcriptions })

rhyming_patterns = defaultdict(list)
for word, ipa_list in word_ipa_map.items():
    for ipa in ipa_list:
        ipa_pattern = ipa[-2:]
        rhyming_patterns[ipa_pattern].append(word)

for pattern, words in rhyming_patterns.items():
    if len(set(words)) > 1:
        print(f"{pattern}:: {', '.join(words)}")

Palabras que terminan con pronunciacion similar
ɣo:: juego, fuego
on:: con, corazón
ʎa:: brilla, orilla


### Práctica 1: Buscador equivalencias fonéticas en un corpus

**Fecha de entrega: Domingo 27 de Agosto 2023 11:59pm**

- Agregar un nuevo modo de búsqueda donde se extienda el comportamiento básico del buscador para ahora buscar por frases. Ejemplo:

```
[es_MX]>>> Hola que hace
 /ola/ /ke/ /ase/
```

- Optimizar el código para agregar los datasets en un "cache" a demanda y no descargar todo el corpus de un trancazo. Esto quiere decir que al inicio de la ejecución no habrá ningun dataset descargado. Mientras la usuaria vaya seleccionado idiomas los irá agregando a un cache local (puede ser persistente o en memoria). Ejemplo:

```
lang>> es_MX
Corpus no encontrado. Descargando ...
[es_MX]>>
...
lang>> en_US
Corpus no encontrado. Descargando ...
[en_US]>>
...
lang>> es_MX
[es_MX]>>
...
```

#### EXTRA

- Mejorar la solución al escenario cuando no se encuentran las palabras en el dataset mostrando palabras similares. Ejemplo:

```
[es_MX]>> pero
No se encontro <<pero>> en el dataset. Palabras aproximadas:
perro /pero/
perno /peɾno/
[es_MX]>>
```

In [35]:
# Funciones necesarias CÓDIGO DE LA AYUDANTÍA
import requests as r
from collections import defaultdict
# Biblioteca de Códigos ISO de lenguajes
lang_codes = {
  "ar":    "Arabic (Modern Standard)",
  "de":    "German",
  "en_UK": "English (Received Pronunciation)",
  "en_US": "English (General American)",
  "eo":    "Esperanto",
  "es_ES": "Spanish (Spain)",
  "es_MX": "Spanish (Mexico)",
  "fa":    "Persian",
  "fi":     "Finnish",
  "fr_FR": "French (France)",
  "fr_QC": "French (Québec)",
  "is":    "Icelandic",
  "ja":    "Japanese",
  "jam":   "Jamaican Creole",
  "km":    "Khmer",
  "ko":    "Korean",
  "ma":    "Malay (Malaysian and Indonesian)",
  "nb":    "Norwegian Bokmål",
  "nl":    "Dutch",
  "or":    "Odia",
  "ro":    "Romanian",
  "sv":    "Swedish",
  "sw":    "Swahili",
  "tts":   "Isan",
  "vi_C":  "Vietnamese (Central)",
  "vi_N":  "Vietnamese (Northern)",
  "vi_S":  "Vietnamese (Southern)",
  "yue":   "Cantonese",
  "zh":    "Mandarin"
}
iso_lang_codes = list(lang_codes.keys())
# Petición a ipa dict
def get_ipa_dict(iso_lang: str) -> dict:
    """Get ipa-dict file from Github

    Parameters:
    -----------
    iso_lang:
        Language as iso code

    Results:
    --------
    dict:
        Dictionary with words as keys and phonetic representation
        as values for a given lang code
    """
    response = r.get(f"https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/{iso_lang}.txt")
    raw_data = response.text.split("\n")
    return response_to_dict(raw_data[:-1])
# Conversión a datos semiestructurados
def response_to_dict(ipa_list: list) -> dict:
    """Parse to dict the list of word-IPA

    Each element of text has the format:
    [WORD][TAB][IPA]

    Parameters
    ----------
    ipa_list: list
        List with each row of ipa-dict raw dataset file

    Returns
    -------
    dict:
        A dictionary with the word as key and the phonetic
        representation as value
    """
    result = {}
    for item in ipa_list:
        item_list = item.split("\t")
        result[item_list[0]] = item_list[1]
    return result
# Palabra a IPA
def get_ipa_transcriptions(word: str, dataset: dict) -> list[str]:
    """Search for word in a given dataset of IPA phonetics

    Given a word this function return the IPA transcriptions

    Parameters:
    -----------
    word: str
        A word to search in the dataset
    dataset: dict
        A dataset for a given language code
    Returns
    -------
    """
    return dataset.get(word.lower(), "NOT FOUND").split(", ")

In [42]:
# Parte 1:
dataset = {}
def get_dataset(lang_code: str) -> bool:
    """Download corpora from ipa-dict github

    Given a list of iso lang codes download available datasets.

    Returns
    -------
    dict
        Lang codes as keys and dictionary with words-transcriptions
        as values
    """
    if lang_code in dataset:
        return True
    if lang_code not in lang_codes:
        return False
    print(f"{lang_codes[lang_code]} no encontrado, iniciando descarga")
    dataset[lang_code] = get_ipa_dict(lang_code)
    return True
def ask_lang() -> str:
    """Ask user an input language

    Given an ISO code of a language it starts the CLI. In case the
    dataset does not include the language it downloads it. In the
    abscense of a response, it stops the CLI

    Returns
    ----------
    bool
        Whether or not should the program continue
    """
    # Valida la existencia del lenguaje
    lang = input("¿Qué lenguaje quieres aprender hoy?\nlang>> ")
    if lang not in lang_codes:
      print("El sistema no incluye a este lenguaje, *crashea*")
      return
    # Verifica el caché
    print(f"Seleccionó {lang_codes[lang]}")
    if get_dataset(lang):
      print(f"{lang_codes[lang]} está listo para usarse")
    return lang

In [43]:
# Parte dos
def ask_query(lang, dataset) -> str:
  query = input(f".  [{lang}]word>> ")
  if not query:
    return
  return sentence_parser(query, dataset)
def sentence_parser(query: str, dataset: dict) -> str:
    """Get transcriptions of all the words of a sentence

    Returns IPA transcriptions of the dataset for each word
    in a sentence, or a Not Found if necessary

    Returns
    ----------
    str
        IPA transcription of the sentence
    """
    words = query.split(' ')
    result = ""
    for w in words:
      transcript = get_ipa_transcriptions(w, dataset)
      result += w + "\n| " + "\n| ".join(transcript) + "\n"
    return result

In [ ]:
# Extra
# TODO: Encontrar la distancia entre dos palabras y mostrar las que caen
#       bajo cierto umbral

In [44]:
# CLI
print(f"Xochi-learning: Aprende la pronunciación de {len(lang_codes)} idiomas diferentes Owo")
print(f"Idiomas disponibles: {(iso_lang_codes)}")
lang = ask_lang()
while lang:
  current_dataset = dataset[lang]
  query = 'placeholder'
  while query:
    query = ask_query(lang, current_dataset)
    print(query) if query else print("Error al capturar la oración")
  lang = ask_lang()

Xochi-learning: Aprende la pronunciación de 29 idiomas diferentes Owo
Idiomas disponibles: ['ar', 'de', 'en_UK', 'en_US', 'eo', 'es_ES', 'es_MX', 'fa', 'fi', 'fr_FR', 'fr_QC', 'is', 'ja', 'jam', 'km', 'ko', 'ma', 'nb', 'nl', 'or', 'ro', 'sv', 'sw', 'tts', 'vi_C', 'vi_N', 'vi_S', 'yue', 'zh']
¿Qué lenguaje quieres aprender hoy?
lang>> es_ES
Seleccionó Spanish (Spain)
Spanish (Spain) no encontrado, iniciando descarga
Spanish (Spain) está listo para usarse
.  [es_ES]word>> mi amor por ti no conoce límites
mi
| /mi/
amor
| /amoɾ/
por
| /poɾ/
ti
| /ti/
no
| /no/
conoce
| /konoθe/
límites
| /ˈlimites/

.  [es_ES]word>> 
Error al capturar la oración
¿Qué lenguaje quieres aprender hoy?
lang>> es_ES
Seleccionó Spanish (Spain)
Spanish (Spain) está listo para usarse
.  [es_ES]word>> 
Error al capturar la oración
¿Qué lenguaje quieres aprender hoy?
lang>> en_UK
Seleccionó English (Received Pronunciation)
English (Received Pronunciation) no encontrado, iniciando descarga
English (Received Pronuncia